In [ ]:
import os
import shutil
import pathlib
import numpy as np
import pandas as pd

run_id = 1
main_folder_path = pathlib.Path("/home/alp/noetic_ws/src/simulation/images")
folder_path = main_folder_path / f"run_{run_id}"
csv_path = folder_path / "cargo_data.csv"
copy_similar = True  # Changed from delete_similar to copy_similar

# Create output folder for non-recurrent frames
output_folder = main_folder_path / "non_recurrent"
output_csv_path = main_folder_path / "cargo_data.csv"

def normalize(q):
    return q / np.linalg.norm(q)

def quat_angle_diff(q1, q2):
    dot = np.clip(np.dot(q1, q2), -1.0, 1.0)
    angle_rad = 2 * np.arccos(abs(dot))
    return np.degrees(angle_rad)

# Read the CSV file
df = pd.read_csv(csv_path)

# Normalize quaternions
quats = df[['cargo_rot_x', 'cargo_rot_y', 'cargo_rot_z', 'cargo_rot_w']].apply(lambda row: normalize(row.values), axis=1)

# Reference quaternion (identity quaternion)
ref_q = normalize(np.array([0.0, 0.0, 0.0, 1.0]))

# Thresholds
abs_threshold = 5.0
seq_threshold = 2.0

# Counters
abs_close_count = 0
seq_close_count = 0

# Lists to track what to copy (keep the diverse/representative frames)
files_to_copy = []
frame_ids_to_keep = set()

i = 0
while i < len(df):
    q = quats[i]
    frame_id = int(df.loc[i, 'frameId'])
    
    # Check if close to reference quaternion
    angle_from_ref = quat_angle_diff(ref_q, q)
    
    if angle_from_ref < abs_threshold:
        abs_close_count += 1
        # Skip this frame entirely (too close to reference)
        i += 1
        continue
    
    # This frame is good - add it to keep list
    fname = f"{frame_id}.jpg"
    files_to_copy.append(fname)
    frame_ids_to_keep.add(frame_id)
    
    # Now check how many subsequent frames are similar to THIS frame (not to each other)
    base_q = q  # This is our reference frame for the sequence
    j = i + 1
    
    while j < len(df):
        next_q = quats[j]
        angle_from_base = quat_angle_diff(base_q, next_q)  # Compare to the base frame, not previous
        
        if angle_from_base < seq_threshold:
            seq_close_count += 1
            j += 1  # Skip this similar frame
        else:
            break  # Found a different frame, stop the sequence
    
    # Move index to the next different frame
    i = j

if copy_similar:
    # Create output directory if it doesn't exist
    output_folder.mkdir(exist_ok=True)
    
    # Copy files to the new folder
    copied_count = 0
    for file_name in files_to_copy:
        source_path = folder_path / file_name
        dest_path = output_folder / file_name
        
        if os.path.exists(source_path):
            shutil.copy2(source_path, dest_path)
            copied_count += 1
        else:
            print(f"Warning: {file_name} not found in source folder")
    
    # Create new CSV with only the frames we're keeping (diverse/representative ones)
    # Filter dataframe to keep only the frames we identified as diverse
    filtered_df = df[df['frameId'].isin(frame_ids_to_keep)]
    
    # Sort by frameId to maintain sequentiality
    filtered_df = filtered_df.sort_values('frameId').reset_index(drop=True)
    
    # Save the filtered CSV
    filtered_df.to_csv(output_csv_path, index=False)
    
    print(f"Created output folder: {output_folder}")
    print(f"Copied {copied_count} image files")
    print(f"Created new CSV with {len(filtered_df)} rows")

print(f"Original total frames: {len(df)}")
print(f"Total close-to-zero frames: {abs_close_count}")
print(f"Total sequentially close frames: {seq_close_count}")
print(f"Total frames to copy (diverse representatives): {len(files_to_copy)}")

if copy_similar:
    print(f"Diverse/representative frames saved to: {output_folder}")
    print(f"New CSV contains {len(filtered_df)} frames (maintaining sequential order)")
else:
    print(f"Would copy {len(files_to_copy)} diverse frames to separate folder")

In [1]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader, Subset
from pytorch_lightning.loggers import TensorBoardLogger

from dataloader import DroneDataset
from model import Model

# TRAINING SCRIPT WITH ANTI-OVERFITTING MEASURES
SEQUENCE_LENGTH = 5
BATCH_SIZE = 16  # Increased batch size for better generalization
NUM_WORKERS = 8
LEARNING_RATE = 5e-4
FPS = 10

# CRITICAL: Use step_size = sequence_length to eliminate data leakage
full_dataset = DroneDataset(
    images_folder="./data/images",
    csv_path="./data/cargo_data.csv",
    sequence_length=SEQUENCE_LENGTH,
    step_size=SEQUENCE_LENGTH  # NO OVERLAP! This is crucial
)

# Split with gaps to prevent any data leakage
total_size = len(full_dataset)
gap = 10  # Additional gap between splits

train_end = int(0.7 * total_size) - gap
val_start = train_end + gap
val_end = val_start + int(0.15 * total_size) - gap
test_start = val_end + gap

train_indices = list(range(0, train_end))
val_indices = list(range(val_start, val_end))
test_indices = list(range(test_start, total_size))

train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)
test_dataset = Subset(full_dataset, test_indices)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

# DataLoaders with augmentation-like effects (different workers)
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,  # Shuffle training data
    num_workers=NUM_WORKERS,
    pin_memory=True,
    drop_last=True  # Drop incomplete batches
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True
)

# Regularized model
model = Model(
    λ_rot=0.8,
    λ_dir=0.5,
    λ_dist=0.1,
    λ_pos=1.5,
    λ_vel=0.3,
    sequence_length=SEQUENCE_LENGTH
)

# Logger
logger = TensorBoardLogger(
    save_dir="lightning_logs", 
    name="anti_overfitting_model",
)

# AGGRESSIVE anti-overfitting callbacks
callbacks = [
    # Save best models
    pl.callbacks.ModelCheckpoint(
        monitor='val_loss',
        mode='min',
        save_top_k=2,  # Keep fewer models
        filename='{epoch}-{val_loss:.4f}',
        dirpath="./checkpoints/regularized_rnn"
    ),
    
    # # Early stopping with tight patience
    # pl.callbacks.EarlyStopping(
    #     monitor='val_loss',
    #     patience=10,  # Stop early if no improvement
    #     mode='min',
    # ),
    
    # Learning rate monitoring
    pl.callbacks.LearningRateMonitor(logging_interval='epoch')
]

# Trainer with validation frequency
trainer = pl.Trainer(
    logger=logger,
    callbacks=callbacks,
    max_epochs=100,  # Hard limit
    log_every_n_steps=1,
    val_check_interval=0.5,  # Validate twice per epoch
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1,
    precision='16-mixed' if torch.cuda.is_available() else '32',
    gradient_clip_val=1.0,  # Gradient clipping
    accumulate_grad_batches=2  # Effective batch size = 32
)

# Train the model
trainer.fit(model, train_loader, val_loader)

# Test the best model
trainer.test(model, test_loader, ckpt_path="best")

/home/alp/noetic_ws/TezLearning/venv/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn(
Using 16bit Automatic Mixed Precision (AMP)


Total samples loaded: 31135
Step size: 5 (controls overlap)
Sequences available: 6227
Train: 4348, Val: 924, Test: 935


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/alp/noetic_ws/TezLearning/venv/lib/python3.13/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/alp/noetic_ws/TezLearning/checkpoints/regularized_rnn exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params | Mode 
---------------------------------------------------------
0 | feature_extractor | Sequential | 21.3 M | train
1 | feature_dropout   | Dropout    | 0      | train
2 | rnn               | RNN

/home/alp/noetic_ws/TezLearning/venv/lib/python3.13/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 0:  50%|████▉     | 135/271 [00:18<00:18,  7.20it/s, v_num=4, val_loss=1.840]

/home/alp/noetic_ws/TezLearning/venv/lib/python3.13/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 12. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Epoch 43:  11%|█▏        | 31/271 [00:03<00:29,  8.13it/s, v_num=4, val_loss=0.500] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
test_results = trainer.test(model, test_loader)


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Subset
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from dataloader import DroneDataset
from model import Model

torch.set_float32_matmul_precision('high')

SEQUENCE_LENGTH = 5  # Number of consecutive frames to use
BATCH_SIZE = 8
NUM_WORKERS = 8
LEARNING_RATE = 1e-4
FPS = 10  # Your video frame rate

full_dataset = DroneDataset(
    images_folder="./data/images",
    csv_path="./data/cargo_data.csv",
    sequence_length=SEQUENCE_LENGTH,
)

total_size = len(full_dataset)
train_end = int(0.7 * total_size)
val_end = train_end + int(0.15 * total_size)
test_indices = list(range(val_end, total_size))
test_dataset = Subset(full_dataset, test_indices)


test_loader = DataLoader(
    test_dataset,  # Using full dataset for testing like in original
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True
)

# Model
model = Model(
    λ_rot=0.8,    # Rotation loss weight
    λ_dir=0.5,    # Direction vector loss weight
    λ_dist=0.1,   # Distance loss weight
    λ_pos=1.5,    # Position loss weight
    λ_vel=0.3,    # Velocity loss weight
    sequence_length=SEQUENCE_LENGTH
)

# Create trainer
trainer = pl.Trainer()

# Load weights from checkpoint file
checkpoint_path = "weights.ckpt"
model = Model.load_from_checkpoint(checkpoint_path)

# Test the model using the loaded weights
trainer.test(model, dataloaders=test_loader)